In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def make_coordinates(image, line_parameters):
    slope, intercept = line_parameters
    y1 = image.shape[0]
    y2 = int(y1*(3/5))
    x1 = int((y1 - intercept)/slope)
    x2 = int((y2 - intercept)/slope)
    return np.array([x1, y1, x2, y2])

def average_slope_intercept(image, lines):
    left_fit = []
    right_fit = []
    for line in lines:
        x1,y1,x2,y2 = line.reshape(4)
        parameters = np.polyfit((x1,x2), (y1,y2), 1)
        slope = parameters[0]
        intercept = parameters[1]
        if slope < 0:
            left_fit.append((slope, intercept))
        else:
            right_fit.append((slope, intercept))

    if not (left_fit and right_fit):
        return None;
    left_fit_average = np.average(left_fit, axis=0)
    right_fit_average = np.average(right_fit, axis=0)
    # print(left_fit_average, right_fit_average)
    left_line = make_coordinates(image, left_fit_average)
    right_line = make_coordinates(image, right_fit_average)
    return np.array([left_line, right_line])

def Canny(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    blur = cv2.GaussianBlur(gray, (21,21), 0)
    canny = cv2.Canny(blur, 50, 150)
    return canny

def display_lines(image, lines):
    line_image = np.zeros_like(image)
    if lines is not None:
        for x1, y1, x2, y2 in lines:
            cv2.line(line_image, (x1,y1), (x2,y2), (255,0,0), 10)
    return line_image

def region_of_interest(image):
    height = image.shape[0]
    polygons = np.array([[(400, height), (900, height), (675,250)]])
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, polygons, 255)
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

def translate_image(image, ax, ay):
    height, width = image.shape[:2]
    translation_matrix = np.float32([ [1,0,ax], [0,1,ay] ])
    img_translation = cv2.warpAffine(image, translation_matrix, (int(height+ax), int(width+ay)))
    return img_translation

cap = cv2.VideoCapture("C:/Users/CG-DTE/Downloads/Mtech 1st sem reference books/M.Tech project/Original_video.avi")

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        corners = cv2.goodFeaturesToTrack(gray,maxCorners=200,qualityLevel=0.01,minDistance=30,blockSize=3)
        corners = np.int0(corners)

        for i in corners:
            x,y = i.ravel()
            #cv2.circle(frame,(x,y),3,255,-1)
        canny_image = Canny(frame)
        cropped_image = region_of_interest(canny_image)
        cv2.imshow("canny", canny_image)
        cv2.imshow("crop", cropped_image)
        lines = cv2.HoughLinesP(cropped_image, 2, np.pi/180, 100, np.array([]), minLineLength=40, maxLineGap=5)
        if lines is not None:
            lines_prev = lines
        else:
            lines = lines_prev

        averaged_lines = average_slope_intercept(frame, lines)
        print(averaged_lines)
        if averaged_lines is None:
            averaged_lines = averaged_lines_prev
        else:
            averaged_lines_prev = averaged_lines
        line_image = display_lines(frame, averaged_lines)

        combo_image = cv2.addWeighted(frame, 0.8, line_image, 1, 1)
        print(frame.shape)
        trans_image = translate_image(frame, (averaged_lines[0][0]+averaged_lines[1][0])/2-frame.shape[1]/2, 0)
        #cv2.imshow("tlate", trans_image)
        cv2.imshow("image",combo_image)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()